In [ ]:
import re
import numpy as np

with open("5/input.txt", "r") as inpt:
    inpt = inpt.read()
    ranges = [[int(d1), int(d2)] for d1, d2 in re.findall(r"^(\d+)-(\d+)$", inpt, re.MULTILINE)]
    items = [int(d) for d in re.findall(r"^\d+$", inpt, re.MULTILINE)]

solution = [range(ranges[0][0], ranges[0][1])]


for start, stop in ranges:
    range_expanded = False
    # Check each solution range
    for i, rng in enumerate(solution):
        if start in rng and stop - 1 not in rng:
            solution[i] = range(rng.start, stop)
            range_expanded = True
            break
        
    if not range_expanded:
        solution.append(range(start, stop))
            
        
print(solution)


In [ ]:

import matplotlib.pyplot as plt
rg = np.transpose(ranges)
fig = plt.figure(figsize=(5, 0.2))

for i, (start, end) in enumerate(ranges):
    plt.hlines(i, start, end, colors='red', linewidth=3)
# plt.yticks(range(len(ranges)), [f'Range {i+1}' for i in range(len(ranges))])
plt.xlabel("Value")
plt.show()

In [ ]:
r1 = range(1, 5)

